<a href="https://colab.research.google.com/github/MeenakshiA730/Dashboard/blob/main/CDash.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install dash

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.0/8.0 MB 39.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 228.0/228.0 kB 18.0 MB/s eta 0:00:00
  Created wheel for stringcase: filename=stringcase-1.2.0-py3-none-any.whl size=3568 sha256=1fb4b860a1a5673d9994e38cb012048aa05702b1ea383711ff3bcd33b40b7418
  Stored in directory: /root/.cache/pip/wheels/b4/33/6d/d0820be98063da218c3206fbad2381cd2db3fbb1a0f0d254b5
Successfully built stringcase
  Attempting uninstall: Werkzeug
    Found existing installation: Werkzeug 3.1.3
    Uninstalling Werkzeug-3.1.3:
      Successfully uninstalled Werkzeug-3.1.3
  Attempting uninstall: Flask
    Found existing installation: Flask 3.1.0
    Uninstalling Flask-3.1.0:
      Successfully uninstalled Flask-3.1.0


In [3]:
!pip install dash pandas scikit-learn plotly


In [5]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score

# ✅ Load Dataset
df = pd.read_csv("/content/creditcard.csv")

# ✅ Handling Missing Values
df.dropna(subset=["Class"], inplace=True)  # Ensure target has no NaNs
df.fillna(df.median(), inplace=True)  # Fill missing feature values with median

# ✅ Feature Engineering
scaler = StandardScaler()
df["Amount"] = scaler.fit_transform(df[["Amount"]])
df.drop(columns=["Time"], inplace=True)  # Remove "Time" column

# ✅ Splitting Data
X = df.drop(columns=["Class"])
y = df["Class"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# ✅ Train Model
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# ✅ Model Evaluation
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f"Model Accuracy: {accuracy:.4f}")
print(report)

# 🔥 **DASHBOARD using Dash**
app = dash.Dash(__name__)

app.layout = html.Div([
    html.H1("Credit Card Fraud Detection Dashboard", style={'textAlign': 'center'}),

    html.Label("Select Feature for Distribution"),
    dcc.Dropdown(
        id="feature-dropdown",
        options=[{"label": col, "value": col} for col in X.columns],
        value="Amount"
    ),

    dcc.Graph(id="feature-distribution"),

    html.H3("Fraud vs Non-Fraud Transactions"),
    dcc.Graph(
        figure=px.histogram(df, x="Class", color="Class", title="Distribution of Fraudulent and Non-Fraudulent Transactions")
    ),
])

@app.callback(
    Output("feature-distribution", "figure"),
    Input("feature-dropdown", "value")
)
def update_histogram(feature):
    fig = px.histogram(df, x=feature, title=f"Distribution of {feature}")
    return fig

if __name__ == '__main__':
     app.run(debug=True, use_reloader=False)   # Run dashboard


Model Accuracy: 0.9996
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     56864
           1       0.97      0.77      0.86        98

    accuracy                           1.00     56962
   macro avg       0.99      0.88      0.93     56962
weighted avg       1.00      1.00      1.00     56962



<IPython.core.display.Javascript object>